In [1]:
import redis, json
import pandas as pd
import utils

In [2]:
with open("../../.KEYS/DATA_SRC_INFO.json", "r") as f:
    storage_info = json.load(f)

In [3]:
redis_ip = storage_info['redis_conn_info']['ip']
redis_port = storage_info['redis_conn_info']['port']

redis_sassion = redis.StrictRedis(host=redis_ip, port=redis_port, db=0)

In [11]:
id_list = pd.read_csv("selected.csv").pid.to_list()
id_records = [{"id": id, "get_date": "20240923"} for id in id_list]

In [14]:
def check_id_in_redis_with_transaction(logger, redis_session, records):
    inited_id_records = []
    with redis_session.pipeline() as pipe:
        try:
            redis_session.watch('compare_id')
            for record in records:
                _id = record.get('id')
                _get_date = record.get('get_date')
                # Redis에서 해당 id의 존재 여부 감시 (트랜잭션 적용)
                pipe.multi()  # 트랜잭션 시작
                pipe.hexists('compare_id', _id)
                exists = pipe.execute()[0]  # hexists 결과 실행 후 받아오기
                if not exists:  # Redis hash에 해당 id가 없으면
                    inited_id_records.append({"id": _id, "get_date": _get_date})
                    logger.info(f"id {_id} is not exist in redis. Added to upload list.")
                else:
                    logger.info(f"id {_id} already exists in redis. This id passed.")
            # 모든 트랜잭션 명령어를 실행 (성공 시)
            pipe.execute()
        
        except redis.WatchError:
            # 만약 다른 클라이언트가 키를 수정하여 트랜잭션이 실패한 경우
            logger.warning("Transaction failed. Retrying the check for ids...")
            return check_id_in_redis_with_transaction(logger, redis_session, records)  # 재시도
        except Exception as e:
            logger.error(f"An error occurred during the Redis transaction: {e}")
        finally:
            redis_session.unwatch()
    
    logger.info(f"Completed checking ids: target ids are {inited_id_records}")
    return inited_id_records


In [ ]:
# record에서 없는 id만 redis로 push
def upload_id_into_redis(logger, redis_session, records):
    for record in records:
        _id = record.get('id')
        _get_date = record.get('get_date')
        if not redis_session.hexists('compare_id', _id):  # Redis hash에 해당 id가 없으면
            redis_session.hset('compare_id', _id, _get_date)  # 해당 id를 해시에 저장
            logger.info(f"id {_id} init into redis.")
        else:
            logger.info(f"id {_id} already exist in redis. set id action dismissed")

In [15]:
result, exist_result = check_id_in_redis_with_transaction(redis_sassion, id_records)